## Introduction to Data Science

## The CRISP-DM approach to Data Science Tasks

_________________________

## I. Business Understanding
                    
#### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ Determine business objectives: Determinar formas de auxiliar um estudante internacional recém-formado a conseguir um bom emprego nos EUA   
+ Assess situation:  Muitos dados disponíveis, inclusive com uma mudança política no meio do caminho o que pode gerar resultados interessantes.
+ Determine data mining goals: Coletar dados sobre status das aplicações, empresa, cargo, salário e localização
+ Produce project plan:  Achar padrões e informações interessantes sobre a questão imigratória e nível salarial/de escolaridade. Se possível, no futuro, tentar interagir com outros datasets (como nacionalidade dos imigrantes, grau de escolaridade, e PPP por estado/cidade) e achar informações relevantes para cada emprego, local e salário.

In [1]:
# Import the os module
import os

import sys
import time
import pickle
import itertools
import pandas as pd
import numpy as np
import pylab
import warnings
import seaborn as sns

#import statsmodels.api as sm
#import statsmodels.formula.api as smf

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import pie
#from matplotlib.font_manager import FontProperties
import seaborn as sns

from IPython.display import display, Image
from IPython.core.interactiveshell import InteractiveShell

#%matplotlib inline
#%matplotlib notebook
#matplotlib.rcdefaults()
#matplotlib.verbose.set_level('silent')

data1 = pd.read_csv("../../h1b_disclosure_data_2015_2019.csv", index_col=False)

## II. Data Understanding  

+ Collect initial data: Conjunto de dados sobre aprovação de vistos para estudantes internacionais nos EUA entre os anos de 2015 e 2019. Contém informações sobre a classificação do visto, empresa empregadora, área e código do cargo, assim como nome, classificação da jornada de trabalho (tempo integral ou meio-período), salário prevalecente para o cargo dada a empresa, cidade e estado do trabalho. 
+ Describe data: 
+ Explore data:  
+ Verify data quality: A maior parte dos dados estava correta e boa suficiente para uso. Alguns cargos tinham o nome social trocado com código social, o que foi corrigido. Muitos cargos também apresentam nomes similares, com diferença de praticamente uma letra entre eles (em especial plurais), que foram agrupados para serem considerados um cargo único.   

In [2]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857216 entries, 0 to 857215
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CASE_NUMBER          857216 non-null  object 
 1   CASE_STATUS          857216 non-null  object 
 2   EMPLOYER_NAME        857195 non-null  object 
 3   SOC_NAME             857210 non-null  object 
 4   SOC_CODE             857212 non-null  object 
 5   JOB_TITLE            857211 non-null  object 
 6   FULL_TIME_POSITION   857215 non-null  object 
 7   PREVAILING_WAGE      857212 non-null  float64
 8   WORKSITE_CITY        857188 non-null  object 
 9   WORKSITE_STATE_ABB   857216 non-null  object 
 10  YEAR                 857216 non-null  int64  
 11  WORKSITE_STATE_FULL  857216 non-null  object 
 12  WORKSITE             857216 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 85.0+ MB


In [3]:
data1.info

<bound method DataFrame.info of                CASE_NUMBER          CASE_STATUS  \
0       I-200-09285-514415            CERTIFIED   
1       I-200-09341-564186  CERTIFIED-WITHDRAWN   
2       I-200-09342-274341            CERTIFIED   
3       I-200-09364-425868            CERTIFIED   
4       I-200-10006-413683            WITHDRAWN   
...                    ...                  ...   
857211  I-200-19053-522403            CERTIFIED   
857212  I-200-18282-647333            CERTIFIED   
857213  I-200-19046-203745               DENIED   
857214  I-200-19059-900658               DENIED   
857215  I-200-19073-908844               DENIED   

                                   EMPLOYER_NAME  \
0                                    IT CATS LLC   
1                       XPERT TECHNOLOGIES, INC.   
2                             ISR INFO WAY, INC.   
3           MAGIC CIRCLE FILMS INTERNATIONAL LLC   
4                          SAPHIRE SOLUTIONS INC   
...                                        

In [4]:
data1.describe()

,PREVAILING_WAGE,YEAR
count,8.572120e+05,857216.000000
mean,9.540074e+04,2016.997778
std,1.755489e+06,1.404460
min,0.000000e+00,2015.000000
25%,5.851000e+04,2016.000000
50%,7.032500e+04,2017.000000
75%,8.636200e+04,2018.000000
max,2.977853e+08,2019.000000


## III. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

In [ ]:
#Correção de nome e códigos trocados

soc_code = '17-2141'
soc_name = 'MECHANICAL ENGINEERS'
data1['SOC_NAME'][397759] = soc_name
data1['SOC_CODE'][397759] = soc_code

In [ ]:
#Correção de nome e códigos trocados

data1[data1["SOC_CODE"] == "SOFTWARE DEVELOPERS, APPLICATIONS"]

dic1 = {"15-1132": "SOFTWARE DEVELOPERS, APPLICATIONS"}
dic2 = {"SOFTWARE DEVELOPERS, APPLICATIONS": "15-1132"}

data1["SOC_CODE"] = data1["SOC_CODE"].replace(dic2) 
data1["SOC_NAME"] = data1["SOC_NAME"].replace(dic1)

In [ ]:
#Correção de nome e códigos trocados

data1[data1["SOC_CODE"] == "COMPUTER SYSTEMS ANALYSTS"]

dic1 = {"15-1121": "COMPUTER SYSTEMS ANALYSTS"}
dic2 = {"COMPUTER SYSTEMS ANALYSTS": "15-1121"}

data1["SOC_CODE"] = data1["SOC_CODE"].replace(dic2) 
data1["SOC_NAME"] = data1["SOC_NAME"].replace(dic1)

In [ ]:
#Correção de nome e códigos trocados

data1[data1["SOC_CODE"] == "OPERATIONS RESEARCH ANALYSTS"]

dic1 = {"15-2031": "OPERATIONS RESEARCH ANALYSTS"}
dic2 = {"OPERATIONS RESEARCH ANALYSTS": "15-2031"}

data1["SOC_CODE"] = data1["SOC_CODE"].replace(dic2) 
data1["SOC_NAME"] = data1["SOC_NAME"].replace(dic1)

In [5]:
#Identificando os estados por caso

states = list(set(data1['WORKSITE_STATE_FULL']))
states_dict = {}
type(states_dict)
count = 1
for i in states:
    states_dict[i] = count
    count +=1

data1['STATE_ID'] = data1['WORKSITE_STATE_FULL'].map(states_dict)
data1['STATE_ID']

0         7
1         7
2         8
3         3
4         3
         ..
857211    3
857212    6
857213    1
857214    3
857215    6
Name: STATE_ID, Length: 857216, dtype: int64

In [ ]:
#Identificando as cidades por caso

cities = list(set(data1['WORKSITE_CITY']))
cities_dict = {}
type(cities_dict)
count = 1
for i in cities:
    cities_dict[i] = count
    count +=1

data1['CITY_ID'] = data1['WORKSITE_CITY'].map(cities_dict)
data1['CITY_ID']

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 